In [ ]:
!pip install tensorflow_recommenders
!pip install scann
from typing import Dict, Text
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.5 MB 5.0 MB/s 
     |████████████████████████████████| 588.3 MB 17 kB/s 
     |████████████████████████████████| 6.0 MB 55.0 MB/s 
     |████████████████████████████████| 439 kB 55.4 MB/s 
     |████████████████████████████████| 1.7 MB 51.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
 

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_path="/content/drive/MyDrive/Colab Notebooks/Msc2/DAT-901"

In [ ]:
tf.__version__

'2.11.0'

In [ ]:
# Create sub datasets : data will be used to train a query model and a candidate model
carts_df = pd.read_csv(f'{main_path}/all_carts.csv', index_col=0)
items_df = pd.read_csv(f'{main_path}/all_libelles.csv', index_col=0)
items_df = items_df.drop_duplicates(subset = "LIBELLE")

In [ ]:
# List of first 10 000 unique ticket id
first_carts = pd.unique(carts_df['TICKET_ID'])
carts_df = carts_df.loc[carts_df['TICKET_ID'].isin(first_carts)]


In [ ]:
# Use tensorflow to convert pandas df to tensor slices
carts = tf.data.Dataset.from_tensor_slices(dict(carts_df)).prefetch(tf.data.AUTOTUNE)
items = tf.data.Dataset.from_tensor_slices(dict(items_df)).prefetch(tf.data.AUTOTUNE)

In [ ]:
for x in carts.take(5).as_numpy_iterator():
    print(x)

{'TICKET_ID': 35592159, 'LIBELLE': b'GD JDM4 PAMPLEMOUSSE FL 200ML'}
{'TICKET_ID': 35592159, 'LIBELLE': b'GD JDM4 PAMPLEMOUSSE FL 200ML'}
{'TICKET_ID': 35592159, 'LIBELLE': b'CR JR PARF BIO.SPE AC.SENT.50ML'}
{'TICKET_ID': 35592159, 'LIBELLE': b'EAU MICELLAIRE 3 THES FL200ML'}
{'TICKET_ID': 35592159, 'LIBELLE': b'GD JDM4 TIARE FL 200ML'}


In [ ]:
for x in items.take(5).as_numpy_iterator():
    print(x)

{'LIBELLE': b'GD JDM4 PAMPLEMOUSSE FL 200ML'}
{'LIBELLE': b'CR JR PARF BIO.SPE AC.SENT.50ML'}
{'LIBELLE': b'EAU MICELLAIRE 3 THES FL200ML'}
{'LIBELLE': b'GD JDM4 TIARE FL 200ML'}
{'LIBELLE': b'EDT  UN MATIN AU JARDIN  100ML  MUGUET'}


In [ ]:
carts = carts.map(lambda x: {
    "LIBELLE": x["LIBELLE"],
    "TICKET_ID": x["TICKET_ID"],
}, num_parallel_calls=tf.data.AUTOTUNE)
items = items.map(lambda x: x["LIBELLE"], num_parallel_calls=tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
tf.random.set_seed(42)
shuffled = carts.shuffle(7500000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(7000000)
test = shuffled.skip(7000000).take(500000)

In [ ]:
libelle_items = items.batch(200)
cart_ids = carts.batch(200).map(lambda x: x["TICKET_ID"])

unique_items = np.unique(np.concatenate(list(libelle_items)))
unique_carts = np.unique(np.concatenate(list(cart_ids)))
cart_id_vocabulary = tf.keras.layers.experimental.preprocessing.IntegerLookup(mask_token=None, vocabulary=unique_carts)
libelles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None, vocabulary=unique_items)

In [ ]:
embedding_dimension = 64

In [ ]:
cart_model = tf.keras.Sequential([cart_id_vocabulary,tf.keras.layers.Embedding(cart_id_vocabulary.vocabulary_size(),embedding_dimension)])
libelle_model = tf.keras.Sequential([libelles_vocabulary,tf.keras.layers.Embedding(libelles_vocabulary.vocabulary_size(),embedding_dimension)])

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=items.batch(200).map(libelle_model)
)

In [ ]:
task = tfrs.tasks.Retrieval(
    metrics=metrics
)

In [ ]:
class LibelleRecModel(tfrs.Model):

  def __init__(self, cart_model, libelle_model):
    super().__init__()
    self.libelle_model: tf.keras.Model = libelle_model
    self.cart_model: tf.keras.Model = cart_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    cart_embeddings = self.cart_model(features["TICKET_ID"])
    libelle_embeddings = self.libelle_model(features["LIBELLE"])
    return self.task(cart_embeddings, libelle_embeddings)

In [ ]:
model = LibelleRecModel(cart_model, libelle_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(
    learning_rate=0.5, 
    initial_accumulator_value=0.1,
    epsilon=1e-07,
))

In [ ]:
cached_train = train.shuffle(7000000).batch(200).cache()
cached_test = test.batch(200).cache()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

In [ ]:
model.fit(cached_train, epochs=10, callbacks=[early_stop])

Epoch 1/10
35000/35000 [==============================] - 3598s 103ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_accuracy: 0.0636 - factorized_top_k/top_10_categorical_accuracy: 0.1182 - factorized_top_k/top_50_categorical_accuracy: 0.2411 - factorized_top_k/top_100_categorical_accuracy: 0.3036 - loss: 958.6636 - regularization_loss: 0.0000e+00 - total_loss: 958.6636
Epoch 2/10
35000/35000 [==============================] - 3697s 106ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1630 - factorized_top_k/top_5_categorical_accuracy: 0.5747 - factorized_top_k/top_10_categorical_accuracy: 0.6702 - factorized_top_k/top_50_categorical_accuracy: 0.8357 - factorized_top_k/top_100_categorical_accuracy: 0.8921 - loss: 429.0633 - regularization_loss: 0.0000e+00 - total_loss: 429.0633
Epoch 3/10
35000/35000 [==============================] - 4010s 115ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2472 - factorized_top_k/top_5_

In [ ]:
model.evaluate(cached_test, return_dict=True)

1228/1228 [==============================] - 215s 112ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0185 - factorized_top_k/top_5_categorical_accuracy: 0.1261 - factorized_top_k/top_10_categorical_accuracy: 0.1929 - factorized_top_k/top_50_categorical_accuracy: 0.3467 - factorized_top_k/top_100_categorical_accuracy: 0.4237 - loss: 1779.2692 - regularization_loss: 0.0000e+00 - total_loss: 1779.2692


{'factorized_top_k/top_1_categorical_accuracy': 0.018544163554906845,
 'factorized_top_k/top_5_categorical_accuracy': 0.1261231154203415,
 'factorized_top_k/top_10_categorical_accuracy': 0.1928544044494629,
 'factorized_top_k/top_50_categorical_accuracy': 0.34666138887405396,
 'factorized_top_k/top_100_categorical_accuracy': 0.42365652322769165,
 'loss': 899.8477172851562,
 'regularization_loss': 0,
 'total_loss': 899.8477172851562}

In [ ]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.cart_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model.libelle_model)))
)

In [ ]:
_, rec = scann_index(np.array([int("35592159")]))
print(f"Recommendations for user 35592159: {rec[0, :5]}")

Recommendations for user 35592159: [b'GD JDM4 PAMPLEMOUSSE FL 200ML' b'GLM JDM YLANG YLANG FP200ML'
 b'SERUM INT JEUNESSE ELX7.9 30ml' b'LAIT DEMAQ 3 THES FL200ML'
 b'GM FRUITS NOIRS 200 ML']


In [ ]:
tf.saved_model.save(scann_index,f'{main_path}/retrieval__model__ticket_id',options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"]))